In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from random import randint
from typing import Callable

In [4]:
class Message:
    def __init__(self, sender_uid: int, initial_values: list, info_levels: list[int], key: int = None):
        self.__initial_values = initial_values.copy()
        self.__info_levels = info_levels.copy()
        self.__sender_uid = sender_uid
        self.__key = key

    def get_info_levels(self) -> list[int]:
        return self.__info_levels.copy()

    def get_initial_values(self) -> list:
        return self.__initial_values.copy()

    def get_sender_uid(self) -> int:
        return self.__sender_uid

    def get_key(self):
        return self.__key


class Process:
    def __init__(self, uid: int, initial_val, process_count: int, default_decision_val: int,
                 prob_func: Callable[[], bool], is_key_generator: bool = False):
        """
        
        :param uid: 
        :param initial_val: initial value of process (must be in decision)
        :param process_count: number of all processes
        :param default_decision_val: default value of decision 
        :param prob_func: probability of not having link failure
        :param is_key_generator: whether to generate a key for process
        """
        self.__key = None
        self.__uid_index = uid - 1  # index of process uid in process list which is their uid - 1
        self.__initial_decision_val = initial_val
        self.__process_count = process_count
        self.__default_decision_val = default_decision_val
        self.__initial_process_values()
        self.__initial_info_levels()
        self.__neighbors = None
        self.__received_messages = []
        self.__is_key_generator = is_key_generator
        self.__prob_func = prob_func

    def __initial_process_values(self):
        self.__process_values = [None] * self.__process_count
        self.__process_values[self.__uid_index] = self.__initial_decision_val

    def __update_initial_values(self, values: list):
        if len(values) != self.__process_count:
            raise ValueError('Number of values must be equal to the number of processes')
        for i in range(len(values)):
            if values[i] is not None:
                self.__process_values[i] = values[i]

    def __initial_info_levels(self):
        self.__info_levels = [-1] * self.__process_count
        self.__info_levels[self.__uid_index] = 0

    def __update_my_info_level(self):
        self.__info_levels[self.__uid_index] = min(self.__info_levels) + 1

    def __update_info_levels(self, info_levels: list[int]):
        if not len(info_levels) == len(self.__info_levels):
            raise ValueError(
                f'information levels must have equal length. {len(self.__info_levels)} and {len(info_levels)} are not equal')
        for i in range(len(info_levels)):
            self.__info_levels[i] = max(info_levels[i], self.__info_levels[i])
        self.__update_my_info_level()

    def set_neighbors(self, neighbors: list):
        if self.__neighbors is None:
            self.__neighbors = neighbors

    def generate_key(self, round_number: int) -> int:
        if not self.__is_key_generator:
            raise ValueError('Only key generator process can generates key')
        self.__key = randint(1, round_number)

    def __set_key_from_msg(self, key: int = None):
        if key is not None:
            self.__key = key

    def get_key(self) -> int:
        return self.__key

    def get_uid(self) -> int:
        return self.__uid_index + 1  # Because we saved uid index (process index) 

    def get_initial_val(self) -> int:
        return self.__initial_decision_val

    def get_information_level(self) -> int:
        return self.__info_levels[self.__uid_index]

    def __generate_message(self) -> Message:
        return Message(self.get_uid(), self.__process_values, self.__info_levels, self.__key)

    def send_messages(self):
        msg = self.__generate_message()
        for p in self.__neighbors:
            can_send = self.__prob_func()
            if can_send:
                p.receive_message(msg)
            else:
                p.receive_message(None)

    def receive_message(self, msg: Message):
        self.__received_messages.append(msg)

    def update_from_messages(self) -> [int, int]:
        link_failures = 0
        successes = 0
        for msg in self.__received_messages:
            if msg is not None:
                successes += 1
                self.__update_single_message(msg)
            else:
                link_failures += 1
        self.__received_messages.clear()
        return successes, link_failures

    def __update_single_message(self, msg: Message):
        self.__set_key_from_msg(msg.get_key())
        self.__update_info_levels(msg.get_info_levels())
        self.__update_initial_values(msg.get_initial_values())

    def get_final_decision(self) -> any:
        if self.__key is None:
            return self.__default_decision_val
        if not self.get_information_level() >= self.__key:
            return self.__default_decision_val
        if len(set(self.__process_values)) > 1:
            return self.__default_decision_val
        return self.__initial_decision_val


class CoordinatedAttackSim:
    def __init__(self, process_count: int, round_num: int, key_selector: int = 1, decisions_list: list | set = None,
                 process_initial_vals: list = None, communication_graph: nx.Graph = None,
                 prob_func: Callable[[], bool] = lambda: randint(0, 9) > 0, simulation_num: int = 100,
                 differ_initial_val_on_round: bool = False):
        """
        
        :param process_count: number of all processes
        :param round_num: number of rounds, r in the algorithm description
        :param key_selector: the process that generates the ky in the beginning
        :param decisions_list: all possible decisions, first one, index zero, is the default (if None, default is [0, 1] 0 for failure, 1 for commit)
        :param process_initial_vals: initial_values of processes, if none, it will be generated uniformly from decision_list
        :param communication_graph: the communication graph, default is complete graph with $process_count$ nodes
        :param prob_func: defining probability function that returns true or false (true for sending message and false for having link failure)
        :param simulation_num: number of simulations
        """
        if decisions_list is None:
            decisions_list = [0, 1]
        self.__process_count = process_count
        self.__set_communication_graph(communication_graph)
        self.__key_selector_index = key_selector - 1  # storing index
        self.__round_num = round_num
        self.__decisions_list = decisions_list.copy()  # first index is the default when we don't have same values, default is zero or not commiting
        self.__initial_process_initial_values(process_initial_vals)
        # self.__initial_process() # we do it on each simulation iteration
        self.__prob_func = prob_func
        self.__simulation_num = simulation_num
        self.__df = None
        self.__differ_initial_val_on_round = differ_initial_val_on_round

    def __initial_process_initial_values(self, process_initial_vals: list):
        if process_initial_vals is None:
            # choose a uniformly distributed number between zero and max decision list index, and set decision of that index as initial decision for that process
            max_decision_index = len(self.__decisions_list) - 1
            process_initial_vals = [randint(0, max_decision_index) for _ in range(self.__process_count)]
        elif len(process_initial_vals) != self.__process_count:
            raise ValueError('Number of values must be equal to the number of processes')
        for initial_dec in set(process_initial_vals):
            if initial_dec not in self.__decisions_list:
                raise ValueError('Initial value must be one of the possible decisions')
        self.__process_initial_values = process_initial_vals

    def __initial_process(self):
        self.__processes = [self.__generate_process(i) for i in range(self.__process_count)]
        self.__set_neighbors()

    def __set_neighbors(self):
        for i in range(len(self.__processes)):
            neighbors = self.__communication_graph.neighbors(i)
            neighbor_processes = [self.__processes[i] for i in neighbors]
            self.__processes[i].set_neighbors(
                neighbor_processes)  # this neighbors function gives all processes index that input process can sends message to

    def __generate_process(self, index: int) -> Process:
        return Process(uid=index + 1, initial_val=self.__process_initial_values[index],
                       process_count=self.__process_count, default_decision_val=self.__decisions_list[0],
                       prob_func=self.__prob_func, is_key_generator=self.__key_selector_index == index)

    def __set_communication_graph(self, communication_graph: nx.Graph):
        if communication_graph is None:
            communication_graph = nx.complete_graph(self.__process_count)
        elif communication_graph.number_of_nodes() != self.__process_count:
            raise ValueError('communication_graph must have equal number of nodes with process count')
        self.__communication_graph = communication_graph.copy()

    def get_communication_graph(self) -> nx.Graph:
        return self.__communication_graph.copy()

    def plot_communication_graph(self, default_color: str = 'yellow', key_selector_color: str = 'orange'):
        nodes = list(self.__communication_graph.nodes())
        nodes_labels = [f'process: {node + 1}' for node in nodes]
        labels_dic = dict(zip(nodes, nodes_labels))
        color_maps = [default_color] * self.__process_count
        color_maps[self.__key_selector_index] = key_selector_color
        nx.draw(self.get_communication_graph(), labels=labels_dic, with_labels=True, node_color=color_maps)

    def __reset(self):
        if self.__differ_initial_val_on_round:
            self.__initial_process_initial_values(None)
        self.__initial_process()

    def simulate(self, log: bool = True) -> pd.DataFrame:
        results = []
        for i in range(self.__simulation_num):
            result_dict = self.__single_simulate()
            if log:
                print(f'sim round: {i + 1}, result:{result_dict}')
            results.append(result_dict)
        self.__df = pd.DataFrame.from_dict(results)
        return self.__df.copy()

    def __single_simulate(self) -> dict[str, str]:
        self.__reset()
        self.__processes[self.__key_selector_index].generate_key(self.__round_num)
        total_received_count = 0
        total_failed_count = 0
        for i in range(self.__round_num):
            received_count, failed_count = self.__simulate_round()
            total_received_count += received_count
            total_failed_count += failed_count
        decisions = [p.get_final_decision() for p in self.__processes]
        info_levels = [p.get_information_level() for p in self.__processes]
        keys = [p.get_key() for p in self.__processes]
        is_correct_answer = self.__check_correct_answer(decisions, self.__process_initial_values.copy(),
                                                        total_failed_count)
        return {'initial-vals': str(self.__process_initial_values),
                'decisions': str(decisions),
                'info-levels': str(info_levels),
                'received-count': str(total_received_count),
                'failed-count': str(total_failed_count),
                'key': str(keys),
                'is-correct-answer': str(is_correct_answer)}

    def __check_correct_answer(self, decisions: list, process_initial_vals: list, total_failed_count: int) -> bool:
        if len(set(decisions)) > 1:
            # if we have violated agreement rule
            return False
        if len(set(process_initial_vals)) == 1 and total_failed_count == 0:
            # if we have started with same initial values, and we had no link failure
            if decisions[0] != process_initial_vals[0]:
                # if we have decided on something else than initial value (everyone decided on same thing otherwise 
                # we would have been returned in the previous if) so, if any of our decision (which is same as all 
                # other decisions, and we know due to above if that we have no link failure) is against any of our 
                # initial values (which are all the same so just checking first one is enough) we have violated the 
                # validity condition
                return False
        return True

    def __simulate_round(self) -> [int, int]:
        total_received = 0
        total_failed = 0
        # sending messages
        for p in self.__processes:
            p.send_messages()
        # receiving messages
        for p in self.__processes:
            received, failed = p.update_from_messages()
            total_received += received
            total_failed += failed
        return total_received, total_failed

In [5]:
sim = CoordinatedAttackSim(3, 3, simulation_num=40, prob_func=lambda: randint(0, 1) > 0,
                           differ_initial_val_on_round=True)

In [6]:
sim.simulate(log=True)

sim round: 1, result:{'initial-vals': '[0, 1, 1]', 'decisions': '[0, 0, 0]', 'info-levels': '[2, 1, 1]', 'received-count': '11', 'failed-count': '7', 'key': '[3, 3, 3]', 'is-correct-answer': 'True'}
sim round: 2, result:{'initial-vals': '[1, 0, 1]', 'decisions': '[0, 0, 0]', 'info-levels': '[1, 1, 0]', 'received-count': '6', 'failed-count': '12', 'key': '[3, 3, 3]', 'is-correct-answer': 'True'}
sim round: 3, result:{'initial-vals': '[0, 1, 1]', 'decisions': '[0, 0, 0]', 'info-levels': '[1, 0, 1]', 'received-count': '6', 'failed-count': '12', 'key': '[2, None, 2]', 'is-correct-answer': 'True'}
sim round: 4, result:{'initial-vals': '[1, 0, 1]', 'decisions': '[0, 0, 0]', 'info-levels': '[1, 1, 0]', 'received-count': '9', 'failed-count': '9', 'key': '[2, 2, 2]', 'is-correct-answer': 'True'}
sim round: 5, result:{'initial-vals': '[0, 0, 1]', 'decisions': '[0, 0, 0]', 'info-levels': '[2, 1, 1]', 'received-count': '10', 'failed-count': '8', 'key': '[2, 2, 2]', 'is-correct-answer': 'True'}
sim

,initial-vals,decisions,info-levels,received-count,failed-count,key,is-correct-answer
0,"[0, 1, 1]","[0, 0, 0]","[2, 1, 1]",11,7,"[3, 3, 3]",True
1,"[1, 0, 1]","[0, 0, 0]","[1, 1, 0]",6,12,"[3, 3, 3]",True
2,"[0, 1, 1]","[0, 0, 0]","[1, 0, 1]",6,12,"[2, None, 2]",True
3,"[1, 0, 1]","[0, 0, 0]","[1, 1, 0]",9,9,"[2, 2, 2]",True
4,"[0, 0, 1]","[0, 0, 0]","[2, 1, 1]",10,8,"[2, 2, 2]",True
5,"[0, 0, 1]","[0, 0, 0]","[1, 1, 1]",8,10,"[1, 1, 1]",True
6,"[0, 1, 0]","[0, 0, 0]","[0, 0, 1]",5,13,"[2, 2, 2]",True
7,"[0, 1, 1]","[0, 0, 0]","[1, 1, 1]",7,11,"[1, 1, 1]",True
8,"[0, 0, 0]","[0, 0, 0]","[0, 1, 1]",10,8,"[1, 1, 1]",True
9,"[1, 0, 1]","[0, 0, 0]","[1, 0, 1]",7,11,"[1, None, 1]",True
